In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [3]:
import pickle
import numpy as np

from math import ceil

In [7]:
# opensmile = '../train_val_fea-2/openSMILEfea.pickle'
openl3 = 'train_val_fea/openl3_sr16_128_512.pickle'
w2v2hub = 'train_val_fea/wav2vec_hubert_fea-2.pickle'
w2v2large = 'train_val_fea/wav2vec_large_robast_fea.pickle'

# openl3 = 'audio_pre-emph_fea/openl3_sr16_128_512.pickle'
# w2v2hub = 'audio_pre-emph_fea/wav2vec_hubert_fea.pickle'
# w2v2large = 'audio_pre-emph_fea/wav2vec_large_robust_fea.pickle'

video = 'full_video_fea/enet_b0_8_best_vgaf_cropped_aligned.pickle'
# video = '/home/hse_student/apsidorova/interpolation_enet_b0_8_best_vgaf_cropped_aligned.pickle'
# video = '/home/avsavchenko/src/emotions-multimodal/faces/ABAW/abaw5/enet_b0_8_best_vgaf_cropped.pickle'

In [8]:
# opensmile_fea = pickle.load(open(opensmile, 'rb'))
openl3_fea = pickle.load(open(openl3, 'rb'))
w2v2hub_fea = pickle.load(open(w2v2hub, 'rb'))
w2v2large_fea = pickle.load(open(w2v2large, 'rb'))

video_fea = pickle.load(open(video, 'rb'))

In [9]:
len(video_fea)

2941546

In [10]:
len(openl3_fea), len(w2v2hub_fea), len(w2v2large_fea), len(video_fea)

(110070, 73470, 73470, 2941546)

In [11]:
for i in openl3_fea.keys():
    print(i)
    break

71-30-1920x1080/00001


In [12]:
audio15_keys = {}
for key in w2v2hub_fea.keys():
    folder, sec = key.split('/')
    
    if folder not in audio15_keys.keys():
        audio15_keys[folder] = []
        
    audio15_keys[folder].append(sec)

In [13]:
audio1_keys = {}
for key in openl3_fea.keys():
    folder, sec = key.split('/')
    
    if folder not in audio1_keys.keys():
        audio1_keys[folder] = []
        
    audio1_keys[folder].append(sec)

In [14]:
video_keys = {}
for key in video_fea.keys():
    folder, frame = key.split('/')
    frame = frame[:-4]
    
    if folder not in video_keys.keys():
        video_keys[folder] = []
        
    video_keys[folder].append(frame)

In [15]:
len(audio15_keys.keys()), len(audio1_keys.keys()), len(video_keys.keys())

(598, 598, 594)

In [16]:
len(audio15_keys.keys()), len(audio1_keys.keys()), len(video_keys.keys())

(598, 598, 594)

In [17]:
all_keys = {
            '1.5': {},
            '1.0': {},
            'video': {}}

for key in video_keys.keys():
    all_keys['video'][key] = sorted(video_keys[key])
    
    if 'left' in key or 'right' in key:
        key = key.replace('_left', '')
        key = key.replace('_right', '')
        
    all_keys['1.5'][key] = sorted(audio15_keys[key])
    all_keys['1.0'][key] = sorted(audio1_keys[key])

In [18]:
features = {}
missed = {}

for key in video_keys.keys():
    frames = all_keys['video'][key]
    
    if ('left' in key
        or
        'right' in key):
        m_key = key.replace('_left', '')
        m_key = m_key.replace('_right', '')
        
    else: m_key = key
    
    for frame in frames:
        local_name = f'{key}/{frame}.jpg'
        
        features[local_name] = {}
        features[local_name]['frame'] = video_fea[local_name]
        
        m15 = str(ceil(int(frame) / 45)).zfill(5)
        m1 = str(ceil(int(frame) / 30)).zfill(5)
        
        lname_au15 = f'{m_key}/{m15}'
        lname_au1 = f'{m_key}/{m1}'
        
        if local_name == '18-24-1920x1080/04831.jpg':
            lname_au1 = '18-24-1920x1080/00161'
        
        try:
#             features[local_name]['opensmile'] = opensmile_fea[lname_au15]
            features[local_name]['openl3'] = openl3_fea[lname_au1]
            features[local_name]['w2v2hub'] = w2v2hub_fea[lname_au15]
            features[local_name]['w2v2large'] = w2v2large_fea[lname_au15]
            
        except:
            missed[key] = 0

In [19]:
missed.keys()

dict_keys([])

In [15]:
len(features)

2941546

In [16]:
with open('features.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)